# 지하철 시간대별 승하차 비율 분석을 통한 주거지/업무지 분포 추정 보고서

## 1. 프로젝트 개요
- 분석 목적 : 지하철역별 출퇴근 시간 승하차 패턴을 분석하여, 서울 시민의 주거지와 업무지의 공간적 분포를 추정하고, 이에 따른 교통 혼잡 개선 방향을 제시
- 데이터 출처 : 서울 열린데이터 광장
- 분석 대상 변수:
    - 07시-08시 승차인원, 07시-08시 하차인원
    - 08시-09시 승차인원, 08시-09시 하차인원
    - 17시-18시 승차인원, 17시-18시 하차인원
    - 18시-19시 승차인원, 18시-19시 하차인원
    - 호선명
    - 지하철역
    - 작업일자

In [5]:
import pandas as pd

df = pd.read_csv("./data/Seoul_Subways_time.csv", encoding='cp949')

# 기본 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74277 entries, 0 to 74276
Data columns (total 52 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   사용월           74277 non-null  int64 
 1   호선명           74277 non-null  object
 2   지하철역          74277 non-null  object
 3   04시-05시 승차인원  74277 non-null  int64 
 4   04시-05시 하차인원  74277 non-null  int64 
 5   05시-06시 승차인원  74277 non-null  int64 
 6   05시-06시 하차인원  74277 non-null  int64 
 7   06시-07시 승차인원  74277 non-null  int64 
 8   06시-07시 하차인원  74277 non-null  int64 
 9   07시-08시 승차인원  74277 non-null  int64 
 10  07시-08시 하차인원  74277 non-null  int64 
 11  08시-09시 승차인원  74277 non-null  int64 
 12  08시-09시 하차인원  74277 non-null  int64 
 13  09시-10시 승차인원  74277 non-null  int64 
 14  09시-10시 하차인원  74277 non-null  int64 
 15  10시-11시 승차인원  74277 non-null  int64 
 16  10시-11시 하차인원  74277 non-null  int64 
 17  11시-12시 승차인원  74277 non-null  int64 
 18  11시-12시 하차인원  74277 non-null  int64 
 19  12시-

In [ ]:
# 데이터 샘플 보기
df.head()

,사용월,호선명,지하철역,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,...,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원,작업일자
0,202505,1호선,동대문,695,16,11411,1920,9784,7385,16724,...,9851,903,3141,8,284,0,0,0,0,20250603
1,202505,1호선,동묘앞,148,3,2859,986,4018,5686,8874,...,4711,135,2029,3,178,0,0,0,0,20250603
2,202505,1호선,서울역,688,67,10783,10374,28788,65184,90177,...,32808,4299,8636,18,174,0,0,0,0,20250603
3,202505,1호선,시청,121,3,1938,5755,3754,27708,7542,...,3883,558,899,0,32,0,0,0,0,20250603
4,202505,1호선,신설동,414,32,9502,2362,10403,9078,22203,...,9707,444,3017,0,0,0,0,0,0,20250603


## 2. 데이터 전처리

In [18]:
missing_count = df.isnull().sum()
#결측치는 없는 것을 확인
missing_count[missing_count > 0]

Series([], dtype: int64)

In [11]:
# 필요없는 데이터 제거

# 출퇴근 시간대의 데이터만 사용
df_keep = ['사용월', '호선명', '지하철역', '07시-08시 승차인원', '07시-08시 하차인원', '08시-09시 승차인원', '08시-09시 하차인원',\
            '17시-18시 승차인원','17시-18시 하차인원','18시-19시 승차인원','18시-19시 하차인원', '작업일자']
subways_df = df[df_keep]
# 최근 5년간의 데이터만 사용
subways_df = subways_df[subways_df['사용월'] >= 202001]

## 3. 탐색적 데이터 분석 (EDA)